NameError: name 'df' is not defined

In [ ]:

import pandas as pd
import numpy as np
import csv
import string

#Data Loading
imdb=pd.read_csv('/content/imdb.csv')
imdb.columns = ["label","text"]
print(imdb.head(5))


data_size = imdb.shape

print(data_size)

imdb_col_names = imdb.columns

print(imdb_col_names)
print(imdb.groupby('label').describe())
print(imdb.head(3))


imdb_target=imdb['label']

print(imdb_target)

from nltk.tokenize import word_tokenize
import nltk
nltk.download('all')


def split_tokens(text):

  message = text.lower()


  word_tokens = word_tokenize(message)

  return word_tokens

imdb['tokenized_message'] = imdb['text'].apply(split_tokens)

from nltk.stem.wordnet import WordNetLemmatizer

def split_into_lemmas(text):

    lemma = []

    lemmatizer = WordNetLemmatizer()

    for word in text:

        a=lemmatizer.lemmatize(word)

        lemma.append(a)

    return lemma



imdb['lemmatized_message'] = imdb['tokenized_message'].apply(split_into_lemmas)



print('Tokenized message:',imdb['tokenized_message'])

print('Lemmatized message:',imdb['lemmatized_message'])



                                                text  label
0  I grew up (b. 1965) watching and loving the Th...      0
1  When I put this movie in my DVD player, and sa...      0
2  Why do people who do not know what a particula...      0
3  Even though I have great interest in Biblical ...      0
4  Im a die hard Dads Army fan and nothing will e...      1
(40000, 2)
Index(['text', 'label'], dtype='object')
        text                                                               
       count unique                                                top freq
label                                                                      
0      20019  19815  This show comes up with interesting locations ...    3
1      19981  19908  Hilarious, clean, light-hearted, and quote-wor...    4
                                                text  label
0  I grew up (b. 1965) watching and loving the Th...      0
1  When I put this movie in my DVD player, and sa...      0
2  Why do people who do not 

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_eng is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_ru is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_r

Tokenized message: 0        [i, grew, up, (, b, ., 1965, ), watching, and,...
1        [when, i, put, this, movie, in, my, dvd, playe...
2        [why, do, people, who, do, not, know, what, a,...
3        [even, though, i, have, great, interest, in, b...
4        [im, a, die, hard, dads, army, fan, and, nothi...
                               ...                        
39995    [``, western, union, '', is, something, of, a,...
39996    [this, movie, is, an, incredible, piece, of, w...
39997    [my, wife, and, i, watched, this, movie, becau...
39998    [when, i, first, watched, flatliners, ,, i, wa...
39999    [why, would, this, film, be, so, good, ,, but,...
Name: tokenized_message, Length: 40000, dtype: object
Lemmatized message: 0        [i, grew, up, (, b, ., 1965, ), watching, and,...
1        [when, i, put, this, movie, in, my, dvd, playe...
2        [why, do, people, who, do, not, know, what, a,...
3        [even, though, i, have, great, interest, in, b...
4        [im, a, die, 

In [22]:
from nltk.corpus import stopwords
imdb.sample(frac=0.1, random_state=42)



def stopword_removal(text):

    stop_words = stopwords.words('english')

    filtered_sentence = []

    filtered_sentence = ' '.join([word for word in text if word not in stop_words])

    return filtered_sentence



imdb['preprocessed_message'] = imdb['lemmatized_message'].apply(stopword_removal)

print('Preprocessed message:',imdb['preprocessed_message'])

Training_data=pd.Series(list(imdb['preprocessed_message']))

Training_label=pd.Series(list(imdb['label']))


from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer

tf_vectorizer = CountVectorizer(ngram_range=(1,2),min_df=(1/len(Training_label)),max_df=0.7)

Total_Dictionary_TDM = tf_vectorizer.fit(Training_data)

message_data_TDM = Total_Dictionary_TDM.transform(Training_data)

from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(ngram_range=(1,2),min_df=(1/len(Training_label)),max_df=0.7)

Total_Dictionary_TFIDF = tfidf_vectorizer.fit(Training_data)

message_data_TFIDF = Total_Dictionary_TFIDF.transform(Training_data)


from sklearn.model_selection import train_test_split#Splitting the data for training and testing

train_data,test_data, train_label, test_label = train_test_split(message_data_TDM,Training_label,test_size=0.1)





Preprocessed message: 0        grew ( b . 1965 ) watching loving thunderbird ...
1        put movie dvd player , sat coke chip , expecta...
2        people know particular time past wa like feel ...
3        even though great interest biblical movie , wa...
4        im die hard dad army fan nothing ever change ....
                               ...                        
39995    `` western union '' something forgotten classi...
39996    movie incredible piece work . explores every n...
39997    wife watched movie plan visit sicily stromboli...
39998    first watched flatliners , wa amazed . necessa...
39999    would film good , gross estimated $ 95,000,000...
Name: preprocessed_message, Length: 40000, dtype: object


In [23]:
seed=9
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
train_data_shape = train_data.shape

test_data_shape = test_data.shape
print("The shape of train  :",train_data_shape)

print("The shape of test data :",train_data_shape)

classifier = SVC(kernel='linear', C=0.025, random_state=seed)

classifier = classifier.fit(train_data,train_label)

target = classifier.predict(test_data)

score = accuracy_score(test_label,target)

print('SVM Classifier : ',score)


with open('output.txt', 'w') as file:
    file.write(str((imdb['tokenized_message'][55],imdb['lemmatized_message'][55])))

from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score

train_data,test_data, train_label, test_label = train_test_split(message_data_TDM,Training_label,test_size=0.2)

train_data_shape = train_data.shape

test_data_shape = test_data.shape

print("The shape of train data:",train_data_shape)



print("The shape of test data:",test_data_shape)

classifier =  SGDClassifier(loss = 'modified_huber', Shuffle=True, random_state=seed)

classifier = classifier.fit(train_data,train_label)

target= classifier.predict(test_data)

score = accuracy_score(test_label,target)

print('SGD classifier : ',score)

with open('output1.txt', 'w') as file:
    file.write(str((imdb['preprocessed_message'][55])))

The shape of train  : (36000, 2527102)
The shape of test data : (36000, 2527102)
SVM Classifier :  0.8955
The shape of train data: (32000, 2527102)
The shape of test data: (8000, 2527102)


TypeError: SGDClassifier.__init__() got an unexpected keyword argument 'Shuffle'